In [1]:
import time
print(time.ctime())

Sat Sep 23 16:43:11 2017


This notebook illustrates the usage of [`pycartan`](https://github.com/TUD-RST/pycartan) - a python package built on top of sympy to facilitate calculations  with differential forms, especially in the field of control theory. Currently, this ipython notebook is the only documentation. For further questions either the source code or the authors should be consulted.


$$
\newcommand{\d}{\mathrm{d}}
\newcommand{\wg}{\wedge}
$$

In [2]:
import pycartan as pc
import sympy as sp
from sympy import sin, cos, exp

# ipython extension to display the content of assignments (saves typing effort and redundancy)
# triggered by special comment ##, see https://github.com/cknoll/displaytools
%load_ext displaytools3

First, some differential forms must be created

In [3]:
# create base 1-forms
(x1, x2, x3), (dx1, dx2, dx3) = pc.setup_objects(3) ##

(x1, x2, x3)

((1)dx1, (1)dx2, (1)dx3)

---


### Wedge product and algebra

Now we can build some basic wedge products

In [4]:
dx1^dx2 ##:
dx2^dx1 ##:
dx1^dx1 ##:
dx2^dx2 ##:
dx2^dx1^dx3 ##:

dx1^dx2  := (1)dx1^dx2

___


dx2^dx1  := (-1)dx1^dx2

___


dx1^dx1  := (0)dx1^dx2

___


dx2^dx2  := (0)dx1^dx2

___


dx2^dx1^dx3  := (-1)dx1^dx2^dx3

___


In [5]:
# a 4-form in 3 variables is always zero
# it is displayed in the follwing way

dx2^dx1^dx3^dx3 ##:

dx2^dx1^dx3^dx3  := (0)dx1^dx1^dx1^dx1

___


Roughly speaking, for fixed $k$ differential $k$-forms can be regarded as a module over the ring of scalar functions. Therefore we can perform the following algebraic operations.

In [6]:
w1 = x3*dx1 + x2**2*sin(x1)*dx2 ##:
w2 = cos(x1)*dx1 - 5*dx2 + x3*dx3 ##:
w1^w2 ##:

w1 := (x3)dx1  +  (x2**2*sin(x1))dx2

---


w2 := (cos(x1))dx1  +  (-5)dx2  +  (x3)dx3

---


w1^w2  := (-x2**2*sin(x1)*cos(x1) - 5*x3)dx1^dx2  +  (x3**2)dx1^dx3  +  (x2**2*x3*sin(x1))dx2^dx3

___


---

**Caveat:** In python the `^`-operator (which is overloaded to perform the wedge product) originally means bitwise XOR and has lower precedence than the addition operators `+` and `-`, see the [python reference](https://docs.python.org/3/reference/expressions.html#operator-precedence).
This leads to the following unintuitive results:

In [7]:
dx1^dx2 + dx2^dx3

(2)dx1^dx2^dx3

In [8]:
# what python actually does:
dx1^(dx2 + dx2)^dx3

(2)dx1^dx2^dx3

There are two ways to overcome this problem: brackets or using `*` not only for scalar multiplication but also for the wedge product.

In [9]:
(dx1^dx2) + (dx2^dx3) ##:
dx1*dx2 + dx2*dx3 ##:

(dx1^dx2) + (dx2^dx3)  := (1)dx1^dx2  +  (1)dx2^dx3

___


dx1*dx2 + dx2*dx3  := (1)dx1^dx2  +  (1)dx2^dx3

___


---

### Hodge star

Background: Let $\omega$ be a $k$-form over the $n$-dimensional vectors space $V$ and let $g(\cdot, \cdot)$ be the scalar product associate with $V$. The Hodge-star-operator maps a $k$-form $\omega$ to a special $(n-k)$-form denoted by $*\omega$ which for every $(n-k)$-form $\eta$ has the property: $\omega \wedge \eta = g(*\omega, \eta)\,\d x_1 \wg \ldots \wg \d x_n$. 

Source: Chapter 1, in Agricola, Friedrich: Global Analysis -Differential Forms in Analysis, Geometry and Physics

See also: https://en.wikipedia.org/wiki/Hodge_isomorphism (slightly less general definition, because for the scalar product index $q=0$ is silently assumed.)

**Implementation status and examples**:

A DiffenrentialForm-object Pycartan has the method `.hodge_star()`, which assumes that the scalar product $g$ is given by the identity matrix. The following examples from [Wikipedia](https://en.wikipedia.org/wiki/Hodge_isomorphism#Three_dimensions) can be reproduced:

In [10]:
dx1.hodge_star() ##:
dx2.hodge_star() ##:
dx3.hodge_star() ##:

dx1.hodge_star()  := (1)dx2^dx3

___


dx2.hodge_star()  := (-1)dx1^dx3

___


dx3.hodge_star()  := (1)dx1^dx2

___


---

### Exterior derivative

A very important operation associated with differential forms is to calculate the exterior derivative (or differential) $\mathrm d \omega$ of a $k$-form $\omega$:

In [11]:
w1 ##:
w1.d

w2 ##:
w2.d ##:

w1  := (x3)dx1  +  (x2**2*sin(x1))dx2

___


w2  := (cos(x1))dx1  +  (-5)dx2  +  (x3)dx3

___


w2.d  := (0)dx1^dx2

___


*Hint:* This syntax is implemented using the property mechanism of python which performs a function call under the hood. The result is a 'ordinary' $k+1$-form which can be used in further calculations:

In [12]:
w1.d^w1 ##:
w1.d + w1*w2

w1.d^w1  := (x2**2*sin(x1))dx1^dx2^dx3

___


(-x2**2*sin(x1)*cos(x1) + x2**2*cos(x1) - 5*x3)dx1^dx2  +  (x3**2 - 1)dx1^dx3  +  (x2**2*x3*sin(x1))dx2^dx3

To calculate the differential of a 0-form (i.e., a scalar function) the basis variables have to be passed:

In [13]:
f = x2*sin(x1)
df = pc.d(f, [x1, x2, x3]) ##:

df := (x2*cos(x1))dx1  +  (sin(x1))dx2

---


Of course we have $\d \circ \d f = 0$ :

In [14]:
df.d

(0)dx1^dx2

Due to the Lemma of Poincaré, if $\d \omega = 0$ it is possible to calculate $\eta$ such that $\omega = \d \eta$. This is done by:

In [15]:
df.integrate() ##:

w2 ##:
w2.d ##:
w2.integrate() ##:

df.integrate()  := x2*sin(x1)

___


w2  := (cos(x1))dx1  +  (-5)dx2  +  (x3)dx3

___


w2.d  := (0)dx1^dx2

___


w2.integrate()  := -5*x2 + x3**2/2 + sin(x1)

___


*Note:* Integration is currentliy only implemented for 1-forms.

### Derived flag

An important algebraic construction is the so called 'derived flag'. A derived flag can be easily calculated with pycartan. However the documentation for that has still to be written

### Vector differential forms

With pycartan it es possible to work with Vector Forms. For now, see the code for (kind of) documentation or feel welcome to ask the authors